# JavaScript Callbacks

## [CustomJS Callbacks](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html)

## [CustomJS for Model Property Events](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html#customjs-for-model-property-events)

`CustomJS` は JavaScript のコード断片を表すコールバック関数を作成する。

- `CustomJS` に与えるコード断片 (`code`) のなかで `cb_obj` は、コッルバックを設定されたモデルに束縛される。たとえば、以下の例では、`cb_obj` は `p.x_range` に該当するモデルを参照する。

- `CustomJS` の引数に与える辞書は、コールバックを設定されたモデル以外のモデルへのアクセスを許すために、それらのモデルを辞書に束縛すう働きをする。たとえば、以下の例で与えられる `source` という名前はデータソースへの参照を与えている。

- `source.change.emit()` は、どこにも説明が書かれていないような気がする。おそらく、`emit()` は `source` が変化したことを BokehJS に通知し、描画の更新を促しているのだろう。なお、JSのコード断片で更新している `source.data` は Python 側で用意した `ColumnDataSource` の内容のレプリカのようである。JSのコードでこの内容を更新してはいるものの、その変更内容は Python には伝わらないらしい。実際、スライダーに Python のコールバックを設定し、`y` の内容を表示してみても、その内容に変化はない。

In [1]:
from bokeh.plotting import figure, Figure, show, output_notebook
from bokeh.models import *
from bokeh.layouts import *

output_notebook()

x = [x*0.005 for x in range(0, 200)]
y = x
print(y[:5])
source = ColumnDataSource(data=dict(x=x, y=y))

callback = CustomJS(args=dict(source=source), code='''
    const data = source.data;
    const f = cb_obj.value;
    const x = data['x'];
    const y = data['y'];
    for (var i = 0; i < x.length; i++) y[i] = Math.pow(x[i], f);
    source.change.emit();
    ''')

def plot(doc):
    chart = Plot(plot_width=400, plot_height=400)
    chart.add_glyph(source, Line(x='x', y='y', line_width=3, line_alpha=.6))

    slider = Slider(start=0.1, end=4, value=1, step=.1, title='power')
    slider.js_on_change('value', callback)
    slider.on_change('value', lambda key, old, new: print(y[:5]))  # Python 側のコールバックにより y の内容の変化を確認
    doc.add_root(gridplot([[slider], [chart]]))

show(plot)

Loading BokehJS ...

[0.0, 0.005, 0.01, 0.015, 0.02]


ところで、上の例ではスライダー Widget の変化を通して、スライダーの値の変化をきっかけとしたインタラクションを実現した。実行時の動作は以下のような感じと思われる。

- Slider モデルの値の変化が `value` イベントを生成
- Slider モデルの `value` イベントが引き金として、そのコールバックが起動される。このコールバック (`callback` 関数) は `.js_on_change` で設定されたもの。
- `callback` 関数はデータソースの内容を更新し、
- BokehJS にデータソースが更新された旨を通知する (`source.change.emit()`)
- BokehJS が画面を描き直す

ところで、 [*CustomJS for Model Property Events*](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html#customjs-for-model-property-events) の説明の冒頭には以下の記述がある：

> These `CustomJS` callbacks can be attached to property change events on any Bokeh model, using the `js_on_change` method of Bokeh models:

Bokeh が扱うモデルは `annotations`, `arrow_heads`, `axes`, `callbacks`, ..., `widgets.tables`, `widgets.widget` と多様である。`gluph`, `layouts`, `plots` なども含まれていることは注目すべきだ。上の例ではスライダーのような HTML ウィジェットが例として扱っていたが、他のモデルの値の更新を利用して `CustomJS` を駆動できるように設計されているのだろう。 **要調査**

## CustomJS for User interaction events

In [15]:
import numpy as np

from bokeh import events

def display_event(div, attributes=[], style='float:left; clear:left; font_size=10pt'):
    "Build a suitable CustomJS to display the current event in the div model."
    print(attributes, style)
    code = '''
    var attrs = %s; var args = [];
    for (let i = 0; i < attrs.length; i++) {
        args.push(attrs[i] + '=' + Number(cb_obj[attrs[i]]).toFixed(2));
        let line = '<span style="float:left; clear:left; font_size=10pt"><b>' +
                   cb_obj.event_name +
                   '</b>(' + args.join(', ') + ')</span>\\n';
        console.log('div.text:', div.text);
        let text = div.text.split('\\n');
        console.log('text:', text);
        let lines = text.split('\\n');
        if (lines.length > 35) lines.shift();
        div.text = lines.join('\\n');
    }''' % (attributes)
    print(code)
    return CustomJS(args=dict(div=div), code=code)

x = np.random.random(size=4000) * 100
y = np.random.random(size=4000) * 100
radii = np.random.random(size=4000) * 1.5
colors = [f"#{r:02}{g:02}{150:02}" for r, g in zip(50+2*x, 30+2*y)]

def plot(doc):
    p = figure(tools="pan,wheel_zoom,zoom_in,zoom_out,reset")
    p.scatter(x, y, radius=np.random.random(size=4000) * 1.5,
              fill_color=colors, fill_alpha=0.6, line_color=None)

    div = Div(width=400, height=p.plot_height, height_policy="fixed")
    button = Button(label="Button", button_type="success")
    layout = column(button, row(p, div))

    ## Events with no attributes
    button.js_on_event(events.ButtonClick, display_event(div)) # Button click
    p.js_on_event(events.LODStart, display_event(div))         # Start of LOD display
    p.js_on_event(events.LODEnd, display_event(div))           # End of LOD display
    
    ## Events with attributes
    point_attributes = ['x', 'y', 'sx', 'sy']                  # Point events
    wheel_attributes = point_attributes + ['delta']            # Mouse wheel event
    pan_attributes = point_attributes + ['delta_x', 'delta_y'] # Pan event
    pinch_attributes = point_attributes + ['scale']            # Pinch event

    point_events = [
        events.Tap, events.DoubleTap, events.Press, events.PressUp,
        events.MouseMove, events.MouseEnter, events.MouseLeave,
        events.PanStart, events.PanEnd, events.PinchStart, events.PinchEnd,
    ]

    for event in point_events:
        p.js_on_event(event, display_event(div, attributes=point_attributes))

    p.js_on_event(events.MouseWheel, display_event(div, attributes=wheel_attributes))
    p.js_on_event(events.Pan,        display_event(div, attributes=pan_attributes))
    p.js_on_event(events.Pinch,      display_event(div, attributes=pinch_attributes))

    doc.add_root(layout)

show(plot)

[] float:left; clear:left; font_size=10pt

    var attrs = []; var args = [];
    for (let i = 0; i < attrs.length; i++) {
        args.push(attrs[i] + '=' + Number(cb_obj[attrs[i]]).toFixed(2));
        let line = '<span style="float:left; clear:left; font_size=10pt"><b>' + cb_obj.event_name + '</b>(' + args.join(', ') + ')</span>\n';
        console.log('div.text:', div.text);
        let text = div.text.split('\n');
        console.log('text:', text);
        let lines = text.split('\n');
        if (lines.length > 35) lines.shift();
        div.text = lines.join('\n');
    }
[] float:left; clear:left; font_size=10pt

    var attrs = []; var args = [];
    for (let i = 0; i < attrs.length; i++) {
        args.push(attrs[i] + '=' + Number(cb_obj[attrs[i]]).toFixed(2));
        let line = '<span style="float:left; clear:left; font_size=10pt"><b>' + cb_obj.event_name + '</b>(' + args.join(', ') + ')</span>\n';
        console.log('div.text:', div.text);
        let text = div.text.spli

['#205.47412944291293118.05576539581135150',
 '#118.78333386739364184.95559339556394150',
 '#128.5413913961824680.15548459425702150',
 '#212.8501488394538577.28079847367974150',
 '#83.9809321606047468.51530583519164150',
 '#154.53829373348978172.74205718352607150',
 '#61.6583740108758474.97009467355562150',
 '#124.9615222813831570.15743390696468150',
 '#82.28123576085588167.88620496573662150',
 '#144.8505276952275641.452358413165165150',
 '#55.10571802710937682.01764873894047150',
 '#151.5368925413599211.69627947768615150',
 '#170.63299708050198204.89692878409292150',
 '#115.8391001829502170.89165274429826150',
 '#224.26059595511398145.93461989410025150',
 '#138.6547078952025632.4622008385152150',
 '#245.04390802837398207.36075661069475150',
 '#127.93813940099672144.78221895524763150',
 '#175.9267273543361592.082174171128150',
 '#88.21098512377432220.57970887309773150',
 '#130.40555284254026216.77162080367154150',
 '#144.81818409198033149.64377279515327150',
 '#142.5163027133996379.584